# WIP

In [8]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.SemanticKernel.Agents.Abstractions, *-*"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, *-*"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, *-*"


#!import config/Settings.cs
#!import plugins/FilesPlugin.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents;

using Kernel = Microsoft.SemanticKernel.Kernel;

Installed Packages Microsoft.SemanticKernel, 1.16.2 Microsoft.SemanticKernel.Agents.Abstractions, 1.16.2-alpha Microsoft.SemanticKernel.Agents.Core, 1.16.2-alpha Microsoft.SemanticKernel.Agents.OpenAI, 1.16.2-alpha

In [10]:
#pragma warning disable SKEXP0001, SKEXP0010

var settings = Settings.LoadFromFile("config/settings.json");

var builder = Kernel.CreateBuilder();

builder.AddAzureOpenAIChatCompletion(settings.model, settings.azureEndpoint, settings.apiKey);

var kernel = builder.Build();

__Agents__

The following code creates the agents.

In [5]:
#pragma warning disable SKEXP0110, SKEXP0001 

var ProgamManager = """
    You are a program manager which will take the requirement and create a plan for creating app. Program Manager understands the 
    user requirements and form the detail documents with requirements and costing. 
""";

ChatCompletionAgent ProgaramManagerAgent =
            new()
            {
                Instructions = ProgamManager,
                Name = nameof(ProgaramManagerAgent),
                Kernel = kernel
            };

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001 

var SoftwareEngineer = """
   You are Software Engieer, and your goal is develop web app using HTML and JavaScript (JS) by taking into consideration all
   the requirements given by Program Manager. 
""";

ChatCompletionAgent SoftwareEngineerAgent =
            new()
            {
                Instructions = SoftwareEngineer,
                Name = nameof(SoftwareEngineerAgent),
                Kernel = kernel
            };

In [3]:
#pragma warning disable SKEXP0110, SKEXP0001 

string Manager = """
    You are manager which will review software engineer code, and make sure all client requirements are completed.
     Once all client requirements are completed, you can approve the request by just responding "approve"
""";

 ChatCompletionAgent ProjectManagerAgent =
            new()
            {
                Instructions = Manager,
                Name = nameof(ProjectManagerAgent),
                Kernel = kernel
            };

Error: (13,26): error CS0103: The name 'kernel' does not exist in the current context

In [17]:
using Microsoft.SemanticKernel.Agents.OpenAI;
using Azure.AI.OpenAI.Assistants;
#pragma warning disable SKEXP0110, SKEXP0001 

string FileManager = """
    You are managing files. 
    If one client wants to read a file or wirte content to one you will do this for them.
""";

var fileManagerKernelBuilder = Kernel.CreateBuilder();
var fileManagerkernel = fileManagerKernelBuilder.Build();

OpenAIAssistantConfiguration config = new(settings.apiKey, settings.azureEndpoint);

 OpenAIAssistantAgent  FileManagerAgent =
           await OpenAIAssistantAgent.CreateAsync(
                kernel: fileManagerkernel,
                config: config,
                new(){
                    Name = nameof(FileManagerAgent),
                    Instructions = FileManager,
                    ModelId = settings.model,
                    EnableCodeInterpreter = true,
                });
                

Error: Azure.RequestFailedException: Invalid URL (POST /v1/assistants)
Status: 404 (model_error)

Content:
{
  "error": {
    "message": "Invalid URL (POST /v1/assistants)",
    "type": "invalid_request_error",
    "param": null,
    "code": null
  }
}

Headers:
apim-request-id: REDACTED
x-ms-region: REDACTED
X-Request-ID: REDACTED
ms-azureml-model-error-reason: REDACTED
ms-azureml-model-error-statuscode: REDACTED
Strict-Transport-Security: REDACTED
azureml-model-session: REDACTED
X-Content-Type-Options: REDACTED
x-envoy-upstream-service-time: REDACTED
x-ms-client-request-id: 93495988-66a7-4121-80e3-29c5ebd71dfb
x-ratelimit-remaining-requests: REDACTED
x-ratelimit-remaining-tokens: REDACTED
Date: Sat, 03 Aug 2024 09:53:13 GMT
Content-Length: 145
Content-Type: application/json

   at Azure.Core.HttpPipelineExtensions.ProcessMessageAsync(HttpPipeline pipeline, HttpMessage message, RequestContext requestContext, CancellationToken cancellationToken)
   at Azure.AI.O

In [ ]:
using System.Threading;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;

#pragma warning disable SKEXP0110, SKEXP0001 
private sealed class ApprovalTerminationStrategy : TerminationStrategy
{
    // Terminate when the final message contains the term "approve"
    protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
        => Task.FromResult(history[history.Count - 1].Content?.Contains("approve", StringComparison.OrdinalIgnoreCase) ?? false);
}



In [ ]:
using Microsoft.SemanticKernel.Agents;
#pragma warning disable SKEXP0110, SKEXP0001 

AgentGroupChat chat =
            new(ProgaramManagerAgent, SoftwareEngineerAgent, ProjectManagerAgent)
            {
                ExecutionSettings =
                    new()
                    {
                        TerminationStrategy =
                            new ApprovalTerminationStrategy()
                            {
                                Agents = [ProjectManagerAgent],
                                MaximumIterations = 6,
                            }
                    }
            };

AgentGroupChatSettings s = new ()

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001 

var input = """
I want to develop calculator app. 
Keep it very simple. And get final approval from manager.
""";

chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, input));
Console.WriteLine($"# {AuthorRole.User}: '{input}'");

await foreach (var content in chat.InvokeAsync())
{
    Console.WriteLine($"# {content.Role} - {content.AuthorName ?? "*"}: '{content.Content}'");
}